# Requirements
## Python Env
- This Notebook requires at least Python 3.6.7
## Required Python Modules
Install these from a terminal using "pip3 install module_name"
- pandas
- numpy
- tqdm
- openpyxl (engine: does not need to be imported)

In [1]:
'''
pandas will be our data manipulation module
'''
import pandas as pd
pd.set_option('display.max_columns', None, 'display.max_rows', 200)

'''
numpy will be our array computing module
'''
import numpy as np

'''
tqdm allows us to easily add progress bars to our processes
'''
from tqdm import tqdm

'''
display will allow us to easily display custom data types like dataframes
'''
from IPython.display import display

'''
built-in python modules
'''
import os
import string
import warnings
warnings.filterwarnings('ignore')

# Category & Sub-Category Spreadsheet

In [2]:
subcat_df = pd.read_excel('cat-subcat.xlsx', engine='openpyxl') # requires openpyxl python module, use pip3 install openpyxl to install in your python3 env

In [3]:
subcat_df = subcat_df[subcat_df.Category.notna()]
subcat_df = subcat_df.drop(columns='Unnamed: 0')
subcat_df

,Category,Sub-Category
0,technology,technology_platform
1,technology,startups
2,technology,startup
3,web_development,services
4,web_development,software
...,...,...
292,nanotech,microfabrication
293,nanotech,molecular_engineering
294,nanotech,molecular_self-assembly
295,nanotech,nanomaterials


In [4]:
subcat_df.iloc[49]['Sub-Category'].split('_')

['application', 'service', 'provider']

The commands below were used to clean the original excel into the above.

In [5]:
# for i in range(0, subcat_df.index.size):
#     subcat_df['Sub-Category'] = subcat_df['Sub-Category'].replace(subcat_df.iloc[i]['Sub-Category'], '_'.join(subcat_df.iloc[i]['Sub-Category'].split()))

In [6]:
# subcat_df.to_excel('cat-subcat.xlsx', engine='openpyxl')

## Converting Category/Sub-Category Dataframe into a Python Dictionary

In [7]:
categories = []
sub_categories = []

for i in range(0, subcat_df.index.size):
    sub_categories.append(subcat_df.iloc[i]['Sub-Category'].replace(u'\u200e', ''))

    if subcat_df.iloc[i]['Category'] not in categories:
        categories.append(subcat_df.iloc[i]['Category'])

## Sub-Category Value Count Dictionary

In [8]:
class Value_Count:
    '''
    This class will keep and maintain and internal dictionary used
    for calculating value counts of sub-categories.
    '''
    def __init__(self):
        self.value_count = {}

        for sc in sub_categories: # this initializes all the sub-categories
                                  # with a value count of 0
            self.value_count[sc] = 0
    
    def check_values(self, word, surrounding):
        '''
        this method takes a word, along with the next 4 words following it, as its 
        parameters and increments either the value count of word or the value count
        of a term found in the surround words self.value_count depending on how many
        times this word is found accross the several sub-categories. returns a boolean
        value that indicates whether a match was found.
        '''
        global subcat_df
        punc = '''!()[]{};:"\,<>./?@#$%^&*_~''' # this will be used to strip words and
                                                # surrounding words of punctuation
        match_found = False # this variable will be updated below and returned at the end
        word = word.lower() # makes word lowercase
        
        surrounding = [x.lower() for x in surrounding] # makes all words lowercase
        # the command below removes all punctuation in surrounding
        surrounding = [x.translate(str.maketrans("","", punc)) for x in surrounding]
        
        # here we separate surrounding into 2-word, 3-word, and 4-word terms used
        # to match sc's with more than one word
        surr_2 = ' '.join(surrounding[:2])
        surr_3 = ' '.join(surrounding[:3])
        surr_4 = ' '.join(surrounding[:4])

        for sc in sub_categories:
            if len(sc.split('_')) > 1: # if there is > 1 word in sc

                if surr_2 == ' '.join(sc.split('_')) or surr_3 == ' '.join(sc.split('_')) or surr_4 == ' '.join(sc.split('_')):
                    # print('MATCH! ~~~ ' + surr_2)
                    match_found = True

                    self.value_count[sc] += 1
                else:
                    for i in sc.split('_'): # iterate through every word
                        if i.lower() == word:
                            # print('MATCH! ~~~ ' + word)
                            match_found = True

                            if word not in self.value_count.keys():
                                if word != 'of' and word != 'and':
                                    try:
                                        self.value_count[word] = 1
                                        subcat_df = subcat_df.append({'Category': cat_subcat(sc)[0], 'Sub-Category': word}, ignore_index=True, verify_integrity=True)
                                    except ValueError: # Duplicate
                                        continue
                            else:
                                self.value_count[word] += 1
                        
            else:
                if sc.lower() == word:
                    # print('MATCH! ~~~ ' + word)

                    if word not in self.value_count.keys():
                        self.value_count[word] = 1
                    else:
                        self.value_count[word] += 1

                    self.value_count[sc] += 1

        return match_found

    def final_count(self):
        '''
        this method removes all entries in self.value_count that had a value count of 0
        '''
        self.value_count = {i: j for i,j in self.value_count.items() if j!=0}

## Method That Maps a Sub-Category To Its Corresponding Category

In [9]:
def cat_subcat(subcat):
    try:
        cat = subcat_df[subcat_df['Sub-Category'] == subcat]['Category'].index[0]
        return (subcat_df.loc[cat, 'Category'], subcat)
    except Exception:
        return (None, None)

%time cat_subcat('green_transportation')

CPU times: user 1.07 ms, sys: 254 µs, total: 1.32 ms
Wall time: 2.46 ms


('clean-tech', 'green_transportation')

# COLUMNS AND INDICES

In [10]:
column_dtypes = {'short_description': object, 'description': object, 'overview': object}

dataframe = pd.read_csv(os.getcwd() + '/objects.csv', dtype=column_dtypes, low_memory=False)
print(f'Columns:\n\t{dataframe.columns}\n\n')
print(f'Indeces:\n\t{dataframe.index}\n')

Columns:
	Index(['id', 'entity_type', 'entity_id', 'parent_id', 'name',
       'normalized_name', 'permalink', 'category_code', 'status', 'founded_at',
       'closed_at', 'domain', 'homepage_url', 'twitter_username', 'logo_url',
       'logo_width', 'logo_height', 'short_description', 'description',
       'overview', 'tag_list', 'country_code', 'state_code', 'city', 'region',
       'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'first_funding_at', 'last_funding_at',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at',
       'last_milestone_at', 'milestones', 'relationships', 'created_by',
       'created_at', 'updated_at'],
      dtype='object')


Indeces:
	RangeIndex(start=0, stop=462651, step=1)



In [11]:
dataframe['category_code'].value_counts()

software            17922
web                 15118
other               13617
ecommerce            9065
games_video          7520
mobile               6862
advertising          6098
consulting           5006
enterprise           4441
biotech              4430
hardware             2951
education            2901
public_relations     2846
network_hosting      2350
search               2182
cleantech            1940
health               1698
finance              1386
social               1310
security             1171
medical              1153
analytics            1022
legal                1012
travel                936
local                 785
news                  768
hospitality           768
semiconductor         696
manufacturing         680
sports                675
music                 581
fashion               563
photo_video           544
transportation        489
real_estate           474
messaging             296
automotive            291
design                281
nonprofit   

# Cleaning Our Data

## Dropping Companies with entity_type != 'Company'

In [12]:
entries_before_drop = dataframe.index.size

dataframe = dataframe[dataframe['entity_type'] == 'Company']

print(str(entries_before_drop - dataframe.index.size) + ' entries were dropped.')

266098 entries were dropped.


## Dropping Companies With Disqualifying status

In [13]:
# entries_before_drop = dataframe.index.size

# dataframe = dataframe.drop(dataframe[dataframe['status'] == 'closed'].index)
# dataframe = dataframe.drop(dataframe[dataframe['status'] == 'acquired'].index)

# print(str(entries_before_drop - dataframe.index.size) + ' entries were dropped.')

# Checking if All Entries Have at Least one of \['overview', 'description', 'short_description', 'tag_list'\]

In [14]:
descriptors = ['short_description', 'description', 'overview', 'tag_list', 'good2go']

dataframe['good2go'] = np.where(dataframe['short_description'].notna() | dataframe['description'].notna() | dataframe['overview'].notna() | dataframe['tag_list'].notna(), True, False)

dataframe[dataframe['good2go'] == False][descriptors]

,short_description,description,overview,tag_list,good2go
27,NaN,NaN,NaN,NaN,False
29,NaN,NaN,NaN,NaN,False
42,NaN,NaN,NaN,NaN,False
70,NaN,NaN,NaN,NaN,False
73,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...
196533,NaN,NaN,NaN,NaN,False
196534,NaN,NaN,NaN,NaN,False
196535,NaN,NaN,NaN,NaN,False
196536,NaN,NaN,NaN,NaN,False


In [15]:
62884/dataframe.index.size

0.31993406358590304

As we can see, 62,884 rows have missing data in all three descriptor columns. This is almost 34.07% of all data, so we will need another algorithm to find the sub-category for the other data. For now, we will divide our dataset into two halves to solve individually.

In [16]:
descriptive_df = dataframe[dataframe['good2go'] == True]
nondes_df = dataframe[dataframe['good2go'] == False]

# Merging Our Descriptive Columns Into One

In [17]:
'''
    - This process is expected to take ~3:02 minutes.
'''
def merge_descriptives(df):
    '''
    This method will concatonate all descriptive columns in an entry
    into a single column called 'info'.
    '''
    info = []

    for i in tqdm(range(0, df.index.size), desc='Merging Columns', unit='entries'):
        text = ''

        if str(df.iloc[i]['overview']) != 'nan':
            text += str(df.iloc[i]['overview'])

        if str(df.iloc[i]['tag_list']) != 'nan':
            text += str(df.iloc[i]['tag_list'])

        if str(df.iloc[i]['description']) != 'nan':
            text += str(df.iloc[i]['description'])

        if str(df.iloc[i]['short_description']) != 'nan':
            text += str(df.iloc[i]['short_description'])
        
        info.append(text)

    return info

descriptive_df['info'] = merge_descriptives(descriptive_df)

Merging Columns: 100%|██████████| 132107/132107 [03:29<00:00, 629.57entries/s]


In [18]:
# for i in range(0, 3):
#     display(descriptive_df.iloc[i]['info'])

# display(descriptive_df.columns)

# Finding Word Relevance In Descriptive Column

In [19]:
'''
    *** TODO: try to implement multithreading again
    *** TODO: split overview columm into smaller workloads and at the end,
              append the results of every section to the sub-categories list.
              we do this because the algorithm slows down dramatically the 
              longer it runs, so if we split it into smaller parts, we decrease
              the risk of this slow down happening. it also makes it so i can better
              run this on my computer, because once i start it, my computer will be 
              extremely slow for 12 continuous hours. this way i can run each segment
              at my convinience in case i need the computing power for something else.

    *** Actual running time was 12h 22min 47s. It started off at 13-17 entries per second,
    but the longer it ran, the worse the running time got. During the last few hours, it
    slowed down to 1-2 entries per second.

    *** On second run, it took 4h 54min 25s. It started off at 13-17 entries per second, but due
    to the shorter running time, it only had a chance to slow down to 3-7 entries per second. I 
    believe the longer running time of the first test was due to not having restarted my laptop
    in a long time.

    *** Third run took 5h 30min 53s. It had about 11 thousand more entries.
'''
def maxcat(ccc):
    '''
    This method will return the category with highest value count in dictionary ccc.
    '''
    values = list(ccc.values())
    keys = list(ccc.keys())

    return keys[values.index(max(values))]

def category_code_eval(h, scs, df):
    '''
    This method will evaluate the status of the category_code
    column in a given entry (index h) and will assign a category_code
    if the current one is null. If it's not null, nothing will be done.
    '''
    if str(df.iloc[h]['category_code']) == 'nan':
        category_code_count = {}

        for sc in scs:  # here we iterate through all the sub-categories
                        # in our final sc list from relevance_matching and
                        # we calculate a value count for every category code
                        # derived from the given sub-category
            cat = cat_subcat(sc)[0]
            if cat != None:
                if cat in category_code_count.keys():
                    category_code_count[cat] += 1
                else:
                    category_code_count[cat] = 1

        # display(category_code_count)
        if len(category_code_count) > 0: # here we pick the category with the
                                         # highest value count and assign it to
                                         # the entry
            df['category_code'][h] = maxcat(category_code_count)

def relevance_matching(overview, subcategories_found):
    '''
    This method operates on a list of descriptions and appends to a list
    all the sub-categories that were matched for every entry in our list
    of descriptions.
    '''
    h = 0   # keeps track of the index of current entry

    for i in tqdm(overview, desc='Matching Entries',unit='entries'):
        # print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n'); print(i, '\n')
        ov_words = i.split()    # separates the current description into a list of its words
        prev_wordset = []   # this will be used to keep a history of the words previously searched
        match = False
        word_count = Value_Count()  # Value_Count will keep a dictionary of the sub-categories matched
                                    # and how many times they were matched

        for j, v in enumerate(ov_words): # here we begin operating on every word
            if v in prev_wordset and match: # if current word was in the previous word set
                continue                    # and match was previously set to true, move on the next word
            
            match = word_count.check_values(v, ov_words[j:j+4]) # here we check_values, passing current
                                                                # word and the 4 succeeding words
            prev_wordset = ov_words[j:j+4] # here we update the previous words

        word_count.final_count() # final_count will update the dictionary stored in Value_Count by
                                 # removing all sub-categories with 0-count
        # display(word_count.value_count)

        # ============================================================================================
        # add word_count.value_count keys to a blank list of sub-categories and then append 
        temp_sc = ''    # stores temp sc for cleaning

        # here we iterate through all the keys in our Value_Count dictionary, which
        # should now only include the sub-categories for whom a match was found
        for key in word_count.value_count.keys():
            temp_key = ''

            if len(key.split('_')) > 1: # if sub-category has more than one word
                for word in key.split('_'): # capitalize and replace _'s with spaces
                    temp_key += word.capitalize()+' '
                
                temp_key = temp_key[:-1]
            else:                      # else just capitalize
                temp_key += key.capitalize()

            # add separator '; ' and append sub-category to our final list
            temp_key += '; '
            temp_sc += temp_key

        temp_sc = temp_sc[:-2] # gets rid of final '; '

        # print(temp_sc + '\n')
        subcategories_found.append(temp_sc) # append final string with
                                            # formatted sub-categories

        # change category_code
        category_code_eval(h, word_count.value_count.keys(), descriptive_df)
        h += 1

    return subcategories_found

# ============================================================================
# ====== RUNNING THE ALGORITHM ===============================================

overview = descriptive_df['info']
subcategories_found = []

%time subcat_column = relevance_matching(overview, subcategories_found)

Matching Entries: 100%|██████████| 132107/132107 [5:30:53<00:00,  6.65entries/s]CPU times: user 5h 18min 6s, sys: 9min 10s, total: 5h 27min 17s
Wall time: 5h 30min 53s



In [20]:
subcat_column[:40]

['Technology Platform; Social Media; Entertainment News; Technology; Proprietary; Social; Online; Platform; Internet',
 'Platform; Internet; Video',
 'Online; Computer; Game; Media; Service',
 'Online Social Networking; Online',
 'Online; Hosting',
 'Website Design; Social Media; Media; Website; Social',
 'Infrastructure; Alternative Energy; Green; Energy; Alternative',
 '',
 'Advertising; Service; Instant; Information; Media',
 'Management; Web-based',
 'Video Chat; Alternative; Video; Chat',
 '',
 'Software; Marketing; Solutions; Service; Virtual; Video; Web',
 'Social Network; Social Networking; Social; News; Service; Networks; Data; Network',
 'Marketing; Social Media; Social; Network; Web; Public; Dedicated; Support; Consumer',
 'Online; Public; Tools; Technology; Support; Community',
 '',
 'Image',
 'Social Media; Ad Exchange; Management; Mobile; Platform; Application; Ad; Business; Media; Social; Based; Money',
 'Exercise; Wearable; Data; Activity; Website; Personal; Interface; 

In [21]:
# descriptive_df = descriptive_df[:1200]
descriptive_df['Sub-Categories'] = subcat_column

In [22]:
descriptive_df.to_csv('submapped.csv')